# Libraries used in this project

In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
import array
import os
import argparse
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestClassifier  
from sklearn.metrics import confusion_matrix, classification_report 
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

# Detect pose on the images 

In [2]:

BODY_PARTS = { "Nose": 0, "Neck": 1, "RShoulder": 2, "RElbow": 3, "RWrist": 4,
               "LShoulder": 5, "LElbow": 6, "LWrist": 7, "RHip": 8, "RKnee": 9,
               "RAnkle": 10, "LHip": 11, "LKnee": 12, "LAnkle": 13, "REye": 14,
               "LEye": 15, "REar": 16, "LEar": 17, "Background": 18 }

POSE_PAIRS = [ ["Neck", "RShoulder"], ["Neck", "LShoulder"], ["RShoulder", "RElbow"],
               ["RElbow", "RWrist"], ["LShoulder", "LElbow"], ["LElbow", "LWrist"],
               ["Neck", "RHip"], ["RHip", "RKnee"], ["RKnee", "RAnkle"], ["Neck", "LHip"],
               ["LHip", "LKnee"], ["LKnee", "LAnkle"], ["Neck", "Nose"], ["Nose", "REye"],
               ["REye", "REar"], ["Nose", "LEye"], ["LEye", "LEar"] ]

width = 368
height = 368
inWidth = width
inHeight = height

net = cv.dnn.readNetFromTensorflow("graph_opt.pb")
thr = 0.2

def poseDetect(frame):
    frameWidth = frame.shape[1]
    frameHeight = frame.shape[0]
    
    net.setInput(cv.dnn.blobFromImage(frame, 1.0, (inWidth, inHeight), (127.5, 127.5, 127.5), swapRB=True, crop=False))
    out = net.forward()
    out = out[:, :19, :, :]  

    assert(len(BODY_PARTS) == out.shape[1])

    points = []
    for i in range(len(BODY_PARTS)):
        
        heatMap = out[0, i, :, :]

        _, conf, _, point = cv.minMaxLoc(heatMap)
        x = (frameWidth * point[0]) / out.shape[3]
        y = (frameHeight * point[1]) / out.shape[2]
        points.append((int(x), int(y)) if conf > thr else None)

    for pair in POSE_PAIRS:
        partFrom = pair[0]
        partTo = pair[1]
        assert(partFrom in BODY_PARTS)
        assert(partTo in BODY_PARTS)

        idFrom = BODY_PARTS[partFrom]
        idTo = BODY_PARTS[partTo]

        if points[idFrom] and points[idTo]:
            cv.line(frame, points[idFrom], points[idTo], (0, 255, 0), 3)
            cv.ellipse(frame, points[idFrom], (3, 3), 0, 0, 360, (0, 0, 255), cv.FILLED)
            cv.ellipse(frame, points[idTo], (3, 3), 0, 0, 360, (0, 0, 255), cv.FILLED)

    t, _ = net.getPerfProfile()

    return frame 

# Here we can get the labels and features of image

In [3]:
poses = ['downdog', 'goddess', 'plank', 'tree', 'warrior2']
train_feature=[]
test_feature=[]
train_labels = []
test_labels=[]
def create_train(DIR,label,feature):
    for person in poses:
        path = os.path.join(DIR, person)
        for img in os.listdir(path):
            img_path = os.path.join(path,img)
            img_array = cv.imread(img_path)
            if img_array is None:
                continue 
            else:
                frame = poseDetect(img_array)
                frame_new=cv.imread(img_path)
                if(frame.shape[2]==frame_new.shape[2]):
                    frame_diff=frame-frame_new
                    feature.append(frame_diff)
                    label.append(person)


# For Train images

In [4]:
pose=create_train(r"C:\Users\ankit\Documents\Python Scripts\Face Recognition\Intern Project\Yoga Train Pose",train_labels,train_feature)
features_train=np.array(train_feature)

<ipython-input-4-821c9f00a191>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  features_train=np.array(train_feature)


# For Test images

In [5]:
test_pose = create_train(r"C:\Users\ankit\Documents\Python Scripts\Face Recognition\Intern Project\Yoga Test Pose",test_labels,test_feature)
features_test=np.array(test_feature)

<ipython-input-5-d32712b7e4b1>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  features_test=np.array(test_feature)


In [6]:
features_train.reshape(len(train_labels),1)
features_test.reshape(len(test_labels),1)

train_data=pd.DataFrame(features_train ,columns=['tr_feature'])
train_labels_data=pd.DataFrame(train_labels ,columns=['tr_label'])

test_data=pd.DataFrame(features_test ,columns=['te_feature'])
test_labels_data=pd.DataFrame(test_labels ,columns=['te_label'])

for i in range(len(train_data['tr_feature'])):
    train_data['tr_feature'][i]=train_data['tr_feature'][i].flatten()
for i in range(len(test_data['te_feature'])):
    test_data['te_feature'][i]=test_data['te_feature'][i].flatten()

# LabelEncoder

In [8]:
label=LabelEncoder()
labels_train=label.fit_transform(train_labels_data['tr_label'])
labels_test=label.fit_transform(test_labels_data['te_label']) 

In [10]:
train=[]
test=[]
for i in range(len(train_data['tr_feature'])):
     train.append(train_data ['tr_feature'][i].std())
for i in range(len(test_data['te_feature'])):
     test.append(test_data['te_feature'][i].std())
        
train=np.array(train)
train=np.reshape(train,(len(labels_train),1))
test=np.array(test)
test=np.reshape(test,(len(labels_test),1))

# RandomForestClassifier

In [11]:
rfc=RandomForestClassifier(n_estimators=300,criterion='entropy',
                             max_features='sqrt',min_samples_leaf=10,random_state=100)
rfc.fit(train,labels_train)
predict=rfc.predict(test)
print(classification_report(labels_test,predict))
print(confusion_matrix(labels_test,predict))


              precision    recall  f1-score   support

           0       0.38      0.49      0.43        97
           1       0.43      0.16      0.24        80
           2       0.38      0.34      0.36       115
           3       0.26      0.12      0.16        69
           4       0.31      0.50      0.38       109

    accuracy                           0.35       470
   macro avg       0.35      0.32      0.31       470
weighted avg       0.35      0.35      0.33       470

[[48  0 28  3 18]
 [12 13  9  3 43]
 [39  2 39  6 29]
 [10  6 11  8 34]
 [19  9 15 11 55]]


In [17]:
rfc.fit(train,labels_train)
predict=rfc.predict(train)
print(classification_report(labels_train,predict))
print(confusion_matrix(labels_train,predict))

              precision    recall  f1-score   support

           0       0.48      0.79      0.59       229
           1       0.57      0.32      0.41       147
           2       0.46      0.30      0.36       239
           3       0.51      0.17      0.25       151
           4       0.47      0.67      0.56       242

    accuracy                           0.48      1008
   macro avg       0.50      0.45      0.43      1008
weighted avg       0.49      0.48      0.45      1008

[[180   1  24   2  22]
 [ 29  47  16   7  48]
 [118   1  71   8  41]
 [ 21  18  17  25  70]
 [ 29  15  28   7 163]]


# KNeighborsClassifier

In [26]:
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(train,labels_train)
predict_knn=knn.predict(test)
print(classification_report(labels_test,predict_knn))
print(confusion_matrix(labels_test,predict_knn))

              precision    recall  f1-score   support

           0       0.40      0.53      0.46        97
           1       0.30      0.24      0.27        80
           2       0.44      0.32      0.37       115
           3       0.19      0.25      0.21        69
           4       0.26      0.26      0.26       109

    accuracy                           0.32       470
   macro avg       0.32      0.32      0.31       470
weighted avg       0.33      0.32      0.32       470

[[51  4 13 13 16]
 [11 19 10 19 21]
 [33 11 37 17 17]
 [11 13  4 17 24]
 [20 16 21 24 28]]


In [27]:
knn.fit(train,labels_train)
predict_rfc=knn.predict(train)
print(classification_report(labels_train,predict_rfc))
print(confusion_matrix(labels_train,predict_rfc))

              precision    recall  f1-score   support

           0       0.60      1.00      0.75       229
           1       1.00      0.88      0.94       147
           2       1.00      0.59      0.74       239
           3       1.00      0.89      0.94       151
           4       1.00      0.92      0.96       242

    accuracy                           0.85      1008
   macro avg       0.92      0.86      0.87      1008
weighted avg       0.91      0.85      0.85      1008

[[229   0   0   0   0]
 [ 17 130   0   0   0]
 [ 99   0 140   0   0]
 [ 17   0   0 134   0]
 [ 19   0   0   0 223]]
